<b>Mixing/blending problem: Portfolio design</b>

In [1]:
# Input data
Bonds = {'A', 'B', 'C', 'D', 'E'}
Type = {'A' : 'Muni', 'B' : 'Agency', 'C' : 'Govn', 'D' : 'Govn', 'E' : 'Muni',}
Quality = {'A' : 2, 'B' : 2, 'C' : 1, 'D' : 1, 'E' : 5}
Maturity = {'A' : 9, 'B' : 15, 'C' : 4, 'D' : 3, 'E' : 2}
Yield = {'A' : 0.086, 'B' : 0.054, 'C' : 0.05, 'D' : 0.044, 'E' : 0.09}

In [2]:
from docplex.mp.model import Model
mdl = Model()

In [3]:
amount = mdl.continuous_var_dict(Bonds, lb = 0, name = "amount")

In [4]:
mdl.maximize(mdl.sum(Yield[b] * amount[b] for b in Bonds))

In [5]:
TotalInvested = mdl.sum(amount[b] for b in Bonds) 
mdl.add_constraint(TotalInvested  <= 10) 
mdl.add_kpi(TotalInvested, 'Total Invested')

DecisionKPI(name=Total Invested,expr=amount_C+amount_A+amount_B+amount_E+amount_D)

In [6]:
#Weighted average quality at most 1.4
mdl.add_constraint(mdl.sum((Quality[b]-1.4)*amount[b] for b in Bonds) <=0)

docplex.mp.LinearConstraint[](-0.400amount_C+0.600amount_A+0.600amount_B+3.600amount_E-0.400amount_D,LE,0)

In [7]:
#Weighted average quality at most 1.4
mdl.add_constraint(mdl.sum((Maturity[b]-5)*amount[b] for b in Bonds) <=0)

docplex.mp.LinearConstraint[](-amount_C+4amount_A+10amount_B-3amount_E-2amount_D,LE,0)

In [8]:
# municipal bonds at most 3
mdl.add_constraint(mdl.sum(amount[b] for b in Bonds if Type[b]=='Muni') <= 3)

docplex.mp.LinearConstraint[](amount_A+amount_E,LE,3)

In [9]:
mdl.solve()

docplex.mp.solution.SolveSolution(obj=0.596727,values={amount_C:7.36364,..

In [10]:
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [11]:
mdl.print_solution()

objective: 0.597
status: OPTIMAL_SOLUTION(2)
  amount_C=7.364
  amount_A=2.182
  amount_E=0.455


In [12]:
# mdl.add_kpi(mdl.sum(amount[b] for b in Bonds), 'Total Invested')

In [13]:
mdl.report()

* model docplex_model1 solved with objective = 0.597
*  KPI: Total Invested = 10.000


In [14]:
AverageQuality = sum(Quality[b]*amount[b].solution_value for b in Bonds)/sum(amount[b].solution_value for b in Bonds)

In [15]:
AverageMaturity = sum(Maturity[b]*amount[b].solution_value for b in Bonds)/sum(amount[b].solution_value for b in Bonds)

In [16]:
mdl.add_kpi(AverageQuality, 'Average Quality')
mdl.add_kpi(AverageMaturity, 'Average Maturity')

DecisionKPI(name=Average Maturity,expr=4.999999999999999)

In [17]:
mdl.report()

* model docplex_model1 solved with objective = 0.597
*  KPI: Total Invested   = 10.000
*  KPI: Average Quality  = 1.400
*  KPI: Average Maturity = 5.000
